In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import statsmodels.api as sm
import statsmodels.formula.api as smf
plt.style.use('seaborn-whitegrid')
from sklearn import datasets
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [2]:
# magic to reload modules automatically
%load_ext autoreload
%autoreload 2

In [3]:
#!pip install stargazer
#conda install -c conda-forge statsmodels

In [4]:
rt = pd.read_excel('rt.xlsx')
msci = pd.read_excel('data/KP.xlsx', skiprows=2)
esg_esg = pd.read_excel('esg_esg.xlsx')
it = pd.read_excel('it.xlsx')
ff = pd.read_excel('ff.xlsx')
esg_esg.rename(columns={'Unnamed: 0':'scores'}, inplace=True)
rt = rt.replace(0, np.nan)
rt.dropna(how='all', axis='index', thresh=4, inplace=True)
rt = rt.replace(np.nan, 0)

In [5]:
pd.set_option('display.max_columns',125) # I want to see all the columns
pd.set_option('display.max_rows',600) 

In [6]:
it = it.loc[:,['Unnamed: 0.1','DAX Index','OMXS30B Index','OMXC25 Index','OMX Index']]

for i in it.columns[1:]:
    it.loc[:,i] = it.loc[:,i].pct_change(1)*100
it.drop(index=[0], inplace=True)
it.rename(columns={'Unnamed: 0.1' : 'date'}, inplace=True)
rt.drop(columns='Unnamed: 0', inplace=True)
ff.drop(columns='Unnamed: 0', inplace=True)
rt = rt.loc[:,['date','port_low','port_mid','port_high']]

In [7]:
msci.drop(index=[0,1,2], inplace=True)
msci.rename(columns = {'Unnamed: 0' : 'date','Unnamed: 1' : 'MSCI_w', 'Unnamed: 2' : 'SPX', 'Unnamed: 3' : 'MSCI_e'}, inplace=True)
msci['date'] = pd.to_datetime(msci['date'], format='%Y%m%d')
for i in msci.columns[1:]:
    msci.loc[:,i] = msci.loc[:,i].pct_change(1)*100
msci.reset_index(inplace = True, drop = True)
msci.drop(index=[0], inplace=True)

In [8]:
rt = rt.merge(ff,on=['date'], how='left')
rt = rt.merge(it,on=['date'], how='left')
rt = rt.merge(msci,on=['date'], how='left')
pd.set_option("display.precision", 3)
rt['RF'] = rt['RF']*100

rt.rename(columns={'Mkt-RF' : 'MktRF'}, inplace=True)

In [9]:
filt = (rt['date'] >= pd.to_datetime('2013-01-01')) & (rt['date'] <= pd.to_datetime('2022-02-28')) 
rt = rt[filt]
rt.dropna(how='all', axis='index', thresh=5, inplace=True)

In [10]:
filt = rt[['port_low','port_mid','port_high','MktRF']].describe()
filt

,port_low,port_mid,port_high,MktRF
count,2273.000,2273.000,2273.000,2273.000
mean,0.078,0.041,0.048,0.029
std,1.049,1.023,1.076,1.015
min,-10.321,-10.736,-11.413,-12.010
25%,-0.403,-0.429,-0.424,-0.440
50%,0.124,0.077,0.084,0.070
75%,0.613,0.576,0.568,0.550
max,7.275,6.682,8.876,8.540


In [11]:
#print(filt.to_latex(index = True, multirow = True))

In [12]:
model1 = smf.ols(formula='port_low ~ MktRF', data=rt).fit() 
model11 = smf.ols(formula='port_low ~ MktRF + SMB + HML', data=rt).fit() 
model111 = smf.ols(formula='port_low ~ MktRF + SMB + HML + RMW + CMA', data=rt).fit() 

In [13]:
model2 = smf.ols(formula='port_mid ~ MktRF', data=rt).fit() 
model22 = smf.ols(formula='port_mid ~ MktRF + SMB + HML', data=rt).fit() 
model222 = smf.ols(formula='port_mid ~ MktRF + SMB + HML + RMW + CMA', data=rt).fit() 

In [14]:
model3 = smf.ols(formula='port_high ~ MktRF', data=rt).fit() 
model33 = smf.ols(formula='port_high ~ MktRF + SMB + HML', data=rt).fit() 
model333 = smf.ols(formula='port_high ~ MktRF + SMB + HML + RMW + CMA', data=rt).fit() 
#print(model3.summary())
#trend = model3.params[0] + model3.params[1]* rt['port_high']
#plt.scatter(rt['port_high'],rt['MktRF'])
#plt.plot(trend, rt['port_high'], 'r');

In [15]:
stargazer = Stargazer([model1, model2, model3, model11, model22, model22, model111, model222, model333])
stargazer.custom_columns(['CAMP low', 'CAMP mid', 'CAMP high', 'FF3 low', 'FF3 mid', 'FF3 high', 'FF5 low', 'FF5 mid', 'FF5 high'], [1, 1, 1, 1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.covariate_order(['Intercept', 'MktRF', 'SMB', 'HML', 'RMW', 'CMA'])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'Intercept': 'alpha','MktRF' : 'beta_1','SMB' : 'beta_2 (SMB)', 'HML' : 'beta_3 (HML)',
                          'RMW' : 'beta_4 (RMW)', 'CMA' : 'beta_5 (CMA)'}) #if you want to change names
HTML(stargazer.render_html())
# Use this guide to addit the table

In [16]:
#print(stargazer.render_latex()) #compile it in LaTeX instead of HTML

In [17]:
def SR(m):
    SR = (m.mean()/m.std())*252**.5
    return SR
sharpe_ratio = []
for i in filt.columns[:3]:
    sharpe_ratio.append(SR(rt[i]))

In [18]:
sharpe_ratio

[1.1784773031043825, 0.6308812742106769, 0.7106697872351895]

In [19]:
betas = [model1.params[1], model2.params[1], model3.params[1]]
def TR(m):
    for j in betas:
        TR = (m.mean()/ j )*252**.5
    return TR
treynor_ratio = []
for i in filt.columns[:3]:
    treynor_ratio.append(TR(rt[i]))

In [20]:
betas

[0.8249481942682456, 0.8457153222240437, 0.9075540812200436]

In [21]:
treynor_ratio

[1.3616931455181347, 0.711237840381233, 0.8426392098475165]